In [1]:
import os
import cv2
import torch
import random
import pandas as pd
from torch.utils.data import Dataset
from torchvision import transforms
from glob import glob
import numpy as np
from pathlib import Path
from loaders.visual_loader import CelebDFVisualDataset
from encoders.visual_encoder import VisualEncoder

In [2]:
# Data path
root_dir = Path(r"D:\florida_coursework\third_sem\multimedia_expert_systems\multimedia_prototype\data\celeb_df")
print(f"Data directory: {root_dir}")
print(f"Exists: {root_dir.exists()}")

Data directory: D:\florida_coursework\third_sem\multimedia_expert_systems\multimedia_prototype\data\celeb_df
Exists: True


In [3]:

test_list_file = root_dir / "List_of_testing_videos.txt"

test_videos = []
with open(test_list_file, "r") as f:
    cnt = 0
    for line in f:
        line = line.strip().strip(", '[]")
        if not line:
            continue
        # Each line format: "1 YouTube-real/00170.mp4"
        try:
            lbl_str, rel_path = line.split(maxsplit=1)
            lbl = int(lbl_str)
            test_videos.append({
                "rel_path": rel_path,
                "label": lbl
            })
        except ValueError as e:
            cnt += 1
            print(f"Skipping line due to parsing error: {line} ({e})")
            continue
    print("no. of skipped lines:" ,cnt)

# convert to DataFrame for easy lookup
test_df = pd.DataFrame(test_videos)

# gather all videos from dataset folders
all_videos = []
for subfolder in ["Celeb-real", "Celeb-synthesis", "YouTube-real"]:
    folder_path = root_dir / subfolder
    for vid in glob(str(folder_path / "*.mp4")):
        rel_path = f"{subfolder}/{Path(vid).name}"

        # Determine if video belongs to test set
        if rel_path in test_df["rel_path"].values:
            tag = "test"
            label = int(test_df.loc[test_df["rel_path"] == rel_path, "label"].values[0])
        else:
            # Assign label based on folder for train/val
            if subfolder in ["Celeb-real", "YouTube-real"]:
                label = 1  # real
            else:
                label = 0  # fake
            tag = "trainval"

        all_videos.append({
            "path": vid,
            "rel_path": rel_path,
            "label": label,
            "tag": tag
        })

df = pd.DataFrame(all_videos)
trainval = df[df["tag"] == "trainval"]
test = df[df["tag"] == "test"]


no. of skipped lines: 0


In [4]:
test.head(5)

,path,rel_path,label,tag
1,D:\florida_coursework\third_sem\multimedia_exp...,Celeb-real/id0_0001.mp4,1,test
11,D:\florida_coursework\third_sem\multimedia_exp...,Celeb-real/id10_0001.mp4,1,test
17,D:\florida_coursework\third_sem\multimedia_exp...,Celeb-real/id10_0007.mp4,1,test
26,D:\florida_coursework\third_sem\multimedia_exp...,Celeb-real/id11_0006.mp4,1,test
28,D:\florida_coursework\third_sem\multimedia_exp...,Celeb-real/id11_0008.mp4,1,test


In [5]:
test_list = [
    {"path": row["path"], "label": row["label"]}
    for _, row in test.iterrows()
]
test_list[0]

{'path': 'D:\\florida_coursework\\third_sem\\multimedia_expert_systems\\multimedia_prototype\\data\\celeb_df\\Celeb-real\\id0_0001.mp4',
 'label': 1}

In [6]:
print("Loading video dataset...")
frames_per_video = 8  # Number of frames to extract per video

dataset = CelebDFVisualDataset(
    root_dir=root_dir,
    frames_per_video=frames_per_video
)

Loading video dataset...


In [7]:
len(dataset)

6529

In [9]:
dataset = CelebDFVisualDataset(
    root_dir=root_dir,
    frames_per_video=8,
    video_list=test_list
)
len(dataset)

518

In [10]:
# Step 3: Test single sample - frame extraction and preprocessing
print("Testing single sample frame extraction...")
print("⏳ This may take 10-30 seconds (face detection on 8 frames)...\n")

sample_frames, sample_label = dataset[0]

print(f"✓ Extracted video frames: {sample_frames.shape}")
print(f"  Expected shape: ({frames_per_video}, 3, 224, 224) - 8 RGB frames")
print(f"  Data range: [{sample_frames.min():.3f}, {sample_frames.max():.3f}]")
print(f"  Label: {sample_label.item()} ({'real' if sample_label.item() == 0 else 'fake'})")

# Check if faces were detected (non-zero tensors)
non_zero_frames = (sample_frames.sum(dim=(1,2,3)) > 0).sum().item()
print(f"  Frames with detected faces: {non_zero_frames}/{frames_per_video}")

if non_zero_frames == 0:
    print("⚠ Warning: No faces detected in this video (all zeros)")
elif non_zero_frames < frames_per_video:
    print(f"⚠ Warning: Only {non_zero_frames} frames had detected faces (padded with zeros)")
else:
    print("✓ All frames successfully extracted with faces!")

Testing single sample frame extraction...
⏳ This may take 10-30 seconds (face detection on 8 frames)...

✓ Extracted video frames: torch.Size([8, 3, 224, 224])
  Expected shape: (8, 3, 224, 224) - 8 RGB frames
  Data range: [0.000, 1.000]
  Label: 1 (fake)
  Frames with detected faces: 8/8
✓ All frames successfully extracted with faces!


In [12]:
# Step 1: Setup and imports
import sys
import torch
from torch.utils.data import DataLoader, Subset
from pathlib import Path
# Step 4: Create DataLoader for batch processing
print("Creating DataLoader...")
test_loader = DataLoader(
    dataset,
    batch_size=2,  # Small batch due to video memory requirements
    shuffle=False,
    num_workers=0  # Use 0 for Windows to avoid multiprocessing issues
)

print(f"✓ Created DataLoader with {len(test_loader)} batches")

# Test one batch (this will take time due to face detection)
print("\n⏳ Loading one batch (may take 20-60 seconds)...")
batch_frames, batch_labels = next(iter(test_loader))

print(f"\nBatch content:")
print(f"  - Frames: {batch_frames.shape}")
print(f"  - Labels: {batch_labels.shape}, values: {batch_labels.tolist()}")
print(f"  - Memory usage: ~{batch_frames.element_size() * batch_frames.nelement() / 1024**2:.2f} MB")

Creating DataLoader...
✓ Created DataLoader with 259 batches

⏳ Loading one batch (may take 20-60 seconds)...

Batch content:
  - Frames: torch.Size([2, 8, 3, 224, 224])
  - Labels: torch.Size([2]), values: [1, 1]
  - Memory usage: ~9.19 MB


In [14]:
# Step 5: Initialize VisualEncoder
print("Loading VisualEncoder (Xception + LSTM + Attention)...")

# Configure to match audio/image encoders
lstm_hidden = 256  # LSTM hidden size
embed_dim = lstm_hidden * 2  # 512 (bidirectional LSTM output)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
encoder = VisualEncoder(
    backbone_name="xception",
    pretrained=True,
    lstm_hidden=lstm_hidden,
    lstm_layers=1,
    bidirectional=True,
    dropout=0.3,
    frame_chunk_size=32
).to(device)

encoder.eval()  # Set to evaluation mode

print(f"✓ VisualEncoder loaded on {device}")
print(f"  Backbone: Xception (pretrained on ImageNet)")
print(f"  Frame feature dim: {encoder.frame_feat_dim}")
print(f"  LSTM hidden: {lstm_hidden} (bidirectional)")
print(f"  Output dimension: {encoder.get_out_dim()}")
print(f"  Total parameters: {sum(p.numel() for p in encoder.parameters()):,}")

Loading VisualEncoder (Xception + LSTM + Attention)...
Using device: cuda
✓ VisualEncoder loaded on cuda
  Backbone: Xception (pretrained on ImageNet)
  Frame feature dim: 2048
  LSTM hidden: 256 (bidirectional)
  Output dimension: 512
  Total parameters: 25,661,225


In [15]:
# Step 6: Generate temporal embeddings for all test batches
print("Generating video embeddings...")
print("⏳ This will take a few minutes (frame extraction + encoding)...\n")

all_embeddings = []
all_labels = []

with torch.no_grad():
    for i, (frames, labels) in enumerate(test_loader):
        print(f"  Processing batch {i+1}/{len(test_loader)}...")
        
        # Move batch to device
        frames = frames.to(device)  # (B, T, C, H, W)
        
        # Generate temporal embeddings
        embeddings = encoder(frames)
        
        # Store results
        all_embeddings.append(embeddings.cpu())
        all_labels.append(labels)
        
        print(f"    Input: {frames.shape} → Embeddings: {embeddings.shape}")

# Concatenate all batches
all_embeddings = torch.cat(all_embeddings, dim=0)
all_labels = torch.cat(all_labels, dim=0)

print(f"\n✓ Final Results:")
print(f"  Total embeddings: {all_embeddings.shape}")
print(f"  Total labels: {all_labels.shape}")
print(f"  Labels distribution: real={(all_labels==0).sum().item()}, fake={(all_labels==1).sum().item()}")
print(f"\n🎉 Video embeddings ready for multimodal fusion!")

Generating video embeddings...
⏳ This will take a few minutes (frame extraction + encoding)...

  Processing batch 1/259...
    Input: torch.Size([2, 8, 3, 224, 224]) → Embeddings: torch.Size([2, 512])
  Processing batch 2/259...
    Input: torch.Size([2, 8, 3, 224, 224]) → Embeddings: torch.Size([2, 512])
  Processing batch 3/259...
    Input: torch.Size([2, 8, 3, 224, 224]) → Embeddings: torch.Size([2, 512])
  Processing batch 4/259...
    Input: torch.Size([2, 8, 3, 224, 224]) → Embeddings: torch.Size([2, 512])
  Processing batch 5/259...
    Input: torch.Size([2, 8, 3, 224, 224]) → Embeddings: torch.Size([2, 512])
  Processing batch 6/259...
    Input: torch.Size([2, 8, 3, 224, 224]) → Embeddings: torch.Size([2, 512])
  Processing batch 7/259...
    Input: torch.Size([2, 8, 3, 224, 224]) → Embeddings: torch.Size([2, 512])
  Processing batch 8/259...
    Input: torch.Size([2, 8, 3, 224, 224]) → Embeddings: torch.Size([2, 512])
  Processing batch 9/259...
    Input: torch.Size([2, 8,

In [16]:
save_path = "embeddings/test_video_embeddings.pt"
torch.save({
    "embeddings": all_embeddings,   # (N, D)
    "labels": all_labels            # (N,)
}, save_path)

print("Saved:", save_path)

Saved: embeddings/test_video_embeddings.pt
